In [1]:
import numpy as np
from datetime import date

In [2]:
# 이자율, 시행횟수 등 변수지정
n = 10000 ; r = 0.0165
x_vol = 0.249 ; y_vol = 0.2182

In [4]:
# 시간 설정 및 체크포인트
n0 = date.toordinal(date(2018, 6, 29))
n1 = date.toordinal(date(2018, 12, 21))
n2 = date.toordinal(date(2019, 6, 25))
n3 = date.toordinal(date(2019, 12, 23))
n4 = date.toordinal(date(2020, 6, 24))
n5 = date.toordinal(date(2020, 12, 23))
n6 = date.toordinal(date(2021, 6, 24))
check_day = np.array([n1-n0, n2-n0, n3-n0,\
                      n4-n0, n5-n0, n6-n0])
rho = 0.0981; corr = np.array([[1,rho],
                              [rho,1]])
oneyear = 365 ; tot_date = n6 - n0 ; dt = 1/oneyear

In [6]:
# cholesky matrix 만들기
k = np.linalg.cholesky(corr)
S1 = np.zeros((tot_date+1, 1))
S2 = np.zeros((tot_date+1, 1))
# 기초자산의 초기 값
S1[0] = 100; S2[0] = 100
ratio_S1 = S1[0] ; ratio_S2 = S2[0]
strike_price = ([0.90, 0.90, 0.85,\
             0.85, 0.80, 0.75])
# 조기상환 횟수
repay_n = len(strike_price)
# 조기상환시 받는 쿠폰 이자율
coupon_rate = ([0.025, 0.05, 0.075, 0.1, 0.125, 0.15])
# 조기상환시 payoff vector
payment = np.zeros([repay_n, 1])
# payoff vector
payoff = np.zeros([repay_n, 1])
# 전체 payoff vector
tot_payoff = np.zeros([repay_n, 1])
#현가 할인 된 payoff vector
discount_payoff = np.zeros([repay_n, 1])

In [7]:
# facevalue, dummy
face_value = 10**4
dummy = 0.15
# knock-in barrier
kib = 0.50

In [8]:
# 조기상환시 페이오프 벡터
for j in range(repay_n):
    payment[j] = face_value*(1+coupon_rate[j])

In [ ]:
# MonteCarlo
for i in range(n):
    # 상관관계가 있는 난수 생성
    w0 = np.random.normal(0, 1, size=[tot_date,2])
    # 난수 행렬 전치
    w0 = np.transpose(w0)
    w = np.matmul(k,w0)
    for j in range(tot_date):
        